<a href="https://colab.research.google.com/github/faranbutt/ETL-Top-100-Economies/blob/main/etl_100B%24_economies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exercise 3:**
# *Develop ETL pipeline to get Countries with GDP(IMF) > 100 Billion $ Economies from this [page](https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)*

ETL pipleline should:


*   Extract
*   Transform
*   Load
*   Log

**Output should be saved in database and as csv**

In [342]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime

In [343]:
log_file = 'etl_project_log.txt'
output_csv = 'Countries_by_GDP.csv'
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_name = 'Countries_by_GDP'
db_name = 'World_Economies.db'
conn = sqlite3.connect(db_name)
sql_query = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
df = pd.DataFrame(columns=["Country",'GDP_USD_millions'])
df

,Country,GDP_USD_millions


In [345]:
def log_progress(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open(log_file,'a') as lf:
    lf.write(f'{timestamp} - {message} \n')

In [346]:
def extract(df,url):
  r = requests.get(url).text
  data = BeautifulSoup(r,'html.parser')
  tables = data.find_all('table')
  table = tables[2]
  rows = table.find_all('tr')
  for row in rows:
    cols = row.find_all('td')
    if len(cols) != 0:
      if cols[0].find('a') is not None and cols[2].contents[0] != '—':
        data_dict = {"Country":cols[0].a.contents[0],
                     "GDP_USD_millions":cols[2].contents[0].replace(',','')
                     }

        df1 = pd.DataFrame(data_dict,index=[0])
        df = pd.concat([df,df1],ignore_index=True)
  return df

In [347]:
def transform(data):
  GDPlist = data['GDP_USD_millions'].tolist()
  GDPlist = [float(x) for x in GDPlist]
  GDPlist = [round(x/1000,2) for x in GDPlist]
  data['GDP_USD_millions'] = GDPlist
  data = data.rename(columns={'GDP_USD_millions':'GDP_USD_billions'})
  return data

In [348]:
def load_to_csv(data,output_csv):
  data.to_csv(output_csv)
def load_to_db(data,conn,table_name):
  data.to_sql(table_name,con=conn,if_exists='replace',index=False)

In [349]:
def run_query(query,conn):
  data = pd.read_sql(query,con=conn)
  return data

In [350]:
log_progress("ETL JOB STARTED!")
log_progress("Extract Phase Started")
extracted_data = extract(df,url)
log_progress("Extract Phase Ended")
extracted_data

,Country,GDP_USD_millions
0,United States,26854599
1,China,19373586
2,Japan,4409738
3,Germany,4308854
4,India,3736882
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


In [351]:
log_progress("Transform Phase Started")
transformed_data = transform(extracted_data)
print("Transformed Data")
print(transformed_data)
log_progress("Transform Phase Ended Phase Ended")

Transformed Data
              Country  GDP_USD_billions
0       United States          26854.60
1               China          19373.59
2               Japan           4409.74
3             Germany           4308.85
4               India           3736.88
..                ...               ...
186  Marshall Islands              0.29
187             Palau              0.26
188          Kiribati              0.25
189             Nauru              0.15
190            Tuvalu              0.07

[191 rows x 2 columns]


In [352]:
log_progress("Load Phase Started")
load_to_csv(transformed_data,output_csv)
load_to_db(transformed_data,conn,table_name)
log_progress("Load Phase Ended")
print("Economies with GDP > 100 Billion $ ")
run_query(sql_query,conn)

Economies with GDP > 100 Billion $ 


,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
64,Kenya,118.13
65,Angola,117.88
66,Oman,104.90
67,Guatemala,102.31


In [354]:
with open('etl_project_log.txt','r') as log:
  data = log.read()
  print(data)

2023-Oct-11-13:14:49 - ETL JOB STARTED! 
2023-Oct-11-13:14:49 - Extract Phase Started 
2023-Oct-11-13:15:20 - Extract Phase Ended 
2023-Oct-11-13:15:28 - Transform Phase Started 
2023-Oct-11-13:15:28 - Transform Phase Ended Phase Ended 
2023-Oct-11-13:15:35 - Load Phase Started 
2023-Oct-11-13:15:35 - Load Phase Ended 

